In [58]:
# Imports and engine

import pandas as pd
import numpy as np
import datetime
from sqlalchemy import create_engine
from cash_flow.util.Converters import date_format

engine = create_engine("sqlite:///data/database.db", echo=False)

In [59]:
# Setup filters
date_from = "2024-01-01"
date_through = "2025-12-31"
report_period = 'W-SUN'

In [60]:
# Load all data from database

bank_df = pd.read_sql_query('SELECT * FROM D13_CF_Bank_Union WHERE d_date >= "' + date_from + '" AND d_date <= "' + date_through + '" ', engine)
cash_df = pd.read_sql_query('SELECT * FROM D10_Cash_Transactions WHERE d_date <= "' + date_through + '" ', engine)
definition_df = pd.read_sql_table("E01_CashFlowDefinition", engine)
definition_df.rename(columns={"id":"definition_id"}, inplace=True)
definition_acc_df = definition_df[definition_df["definition_type"] == 1]
definition_tot_df = definition_df[definition_df["definition_type"] == 2]
definition_bal_df = definition_df[definition_df["definition_type"] == 3]
definition_accounts_df = pd.read_sql_table("E01_CashFlowDefinitionAccounts", engine)
definition_totals_df = pd.read_sql_table("E01_CashFlowDefinitionTotals", engine)


In [61]:
bank_df.head()

,d_id,d_type,d_date,d_customer,d_vendor,gl_account,gl_entry_type,gl_amount,d_currency,gl_amount_LC
0,1502,1,2025-05-01 00:00:00.000000,62.0,NaN,2310,DR,0.00,EUR,0.00
1,1502,1,2025-05-01 00:00:00.000000,62.0,NaN,2620,DR,3386.69,EUR,3386.69
2,1502,1,2025-05-01 00:00:00.000000,62.0,NaN,5721,CR,587.77,EUR,587.77
3,1502,1,2025-05-01 00:00:00.000000,62.0,NaN,6110,CR,2798.92,EUR,2798.92
4,1504,1,2025-09-16 00:00:00.000000,67.0,NaN,2310,DR,0.00,EUR,0.00


In [62]:
cash_df.head()

,id,d_id,d_type,d_number,d_date,d_vendor,d_customer,d_description,d_currency,gl_entry_type,gl_account,gl_amount,gl_amount_LC
0,7568,2284,1,1784,2020-01-01 00:00:00.000000,NaN,45.0,Payment 1784,EUR,DR,2620,1349.31,1349.31
1,7550,2275,1,1775,2020-01-02 00:00:00.000000,NaN,5.0,Payment 1775,EUR,DR,2620,1824.39,1824.39
2,16025,4713,2,3413,2020-01-02 00:00:00.000000,98.0,NaN,Payment 3413,EUR,CR,2620,5267.30,5267.30
3,16959,5180,2,3880,2020-01-02 00:00:00.000000,89.0,NaN,Payment 3880,EUR,CR,2620,753.54,753.54
4,15719,4560,2,3260,2020-01-04 00:00:00.000000,83.0,NaN,Payment 3260,EUR,CR,2620,9757.83,9757.83


In [63]:
definition_acc_df.head()

,definition_id,key,definition_type,name
1,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas
2,3,102,1.0,Maksājumi piegādātājiem un darbuzņēmējiem
3,4,103,1.0,Maksājumi darbiniekiem un pašnodarbinātajiem
4,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi
6,7,201,1.0,Izdevumi procentu maksājumiem


In [64]:
definition_accounts_df.head()

,id,definition_id,operator,entry_type,account
0,1,9,-,CR,6110
1,2,2,+,CR,6110
2,3,3,-,DR,7110
3,4,3,-,DR,7210
4,5,3,-,DR,7310


In [65]:
definition_tot_df.head()

,definition_id,key,definition_type,name
5,6,199,2.0,Bruto pamatdarbības naudas plūsma
9,10,299,2.0,Pamatdarbības neto naudas plūsma
19,20,309,2.0,Ieguldījumu darbības neto naudas plūsma
26,27,499,2.0,Finansēšanas darbības neto naudas plūsma
28,29,902,2.0,Neto naudas plūsma


In [66]:
definition_totals_df.head()

,id,definition_id,operator,definition_summarized
0,1,6,+,2
1,2,6,+,3
2,3,6,+,4
3,4,6,+,5
4,5,10,+,2


In [67]:
definition_bal_df.head()

,definition_id,key,definition_type,name
29,30,999,3.0,Naudas līdzekļu atlikums pārskata perioda beigās
30,31,000,3.0,Naudas līdzekļu atlikums pārskata perioda beigās


In [68]:
# ********************** (1) Start working with sums based on filtered accounts *******************************
# Merge accounts into accounts definition

definition_accounts_df = pd.merge(
    definition_acc_df,
    definition_accounts_df,
    on="definition_id", how="left")
definition_accounts_df.drop(columns=["id"], inplace=True)

In [69]:
definition_accounts_df

,definition_id,key,definition_type,name,operator,entry_type,account
0,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,+,CR,6110
1,3,102,1.0,Maksājumi piegādātājiem un darbuzņēmējiem,-,DR,7110
2,3,102,1.0,Maksājumi piegādātājiem un darbuzņēmējiem,-,DR,7210
3,3,102,1.0,Maksājumi piegādātājiem un darbuzņēmējiem,-,DR,7310
4,4,103,1.0,Maksājumi darbiniekiem un pašnodarbinātajiem,NaN,NaN,NaN
5,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,NaN,NaN,NaN
6,7,201,1.0,Izdevumi procentu maksājumiem,NaN,NaN,NaN
7,8,202,1.0,Izdevumi nodokļu maksājumiem,NaN,NaN,NaN
8,9,203,1.0,Naudas plūsma no ārkārtas posteņiem,-,CR,6110
9,12,301,1.0,Radniecīgo vai asociēto uzņēmumu daļu iegāde,NaN,NaN,NaN


In [70]:
# Merge accounts definition into bank transactions

merged_df = pd.merge(definition_accounts_df,
    bank_df,
    left_on=['entry_type', 'account'],        # Columns in definition_df
    right_on=['gl_entry_type', 'gl_account'], # Corresponding columns in transactions_df
    how='left'
)


In [71]:
# Add column adjusted_amount_LC based on operator + / -

merged_df['adjusted_amount_LC'] = np.where(
    merged_df['operator'] == '+', 
    merged_df['gl_amount_LC'], 
    -merged_df['gl_amount_LC']
)


In [72]:
# Convert d_date to_datetime if not done yet

merged_df['d_date'] = pd.to_datetime(merged_df['d_date'])

# Convert adjusted_amount_LC to_numeric if not done yet

merged_df['adjusted_amount_LC'] = pd.to_numeric(merged_df['adjusted_amount_LC'], errors='coerce')

In [73]:
# Add column for period reference

merged_df["d_period"] = merged_df['d_date'].dt.to_period(report_period).apply(lambda r: r.to_timestamp(how='end').normalize() if pd.notna(r) else pd.NaT)

In [74]:
merged_df.head()

,definition_id,key,definition_type,name,operator,entry_type,account,d_id,d_type,d_date,d_customer,d_vendor,gl_account,gl_entry_type,gl_amount,d_currency,gl_amount_LC,adjusted_amount_LC,d_period
0,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,+,CR,6110,1502.0,1.0,2025-05-01,62.0,NaN,6110,CR,2798.92,EUR,2798.92,2798.92,2025-05-04
1,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,+,CR,6110,1513.0,1.0,2024-08-03,9.0,NaN,6110,CR,3671.18,EUR,3671.18,3671.18,2024-08-04
2,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,+,CR,6110,1514.0,1.0,2025-11-07,18.0,NaN,6110,CR,953.22,EUR,953.22,953.22,2025-11-09
3,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,+,CR,6110,1516.0,1.0,2024-11-15,54.0,NaN,6110,CR,1335.17,EUR,1335.17,1335.17,2024-11-17
4,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,+,CR,6110,1529.0,1.0,2024-12-28,66.0,NaN,6110,CR,3198.61,EUR,3198.61,3198.61,2024-12-29


In [75]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   definition_id       667 non-null    int64         
 1   key                 667 non-null    object        
 2   definition_type     667 non-null    float64       
 3   name                667 non-null    object        
 4   operator            649 non-null    object        
 5   entry_type          649 non-null    object        
 6   account             649 non-null    object        
 7   d_id                649 non-null    float64       
 8   d_type              649 non-null    float64       
 9   d_date              649 non-null    datetime64[ns]
 10  d_customer          346 non-null    float64       
 11  d_vendor            303 non-null    float64       
 12  gl_account          649 non-null    object        
 13  gl_entry_type       649 non-null    object        

In [76]:
# Pivot values based on definition_id and d_period

pivot_df = merged_df.pivot_table(
    index='definition_id',
    columns='d_period',
    values='adjusted_amount_LC',
    aggfunc="sum",
    fill_value=0,
    dropna=False
)


In [77]:
# Add all periods in range if some are missing

all_periods = pd.date_range(start=date_from, end=date_through, freq=report_period)
pivot_df = pivot_df.reindex(columns=all_periods, fill_value=0)


In [78]:
# Ensure columns are in datetime format before formatting
pivot_df.columns = pd.to_datetime(pivot_df.columns, errors='coerce')



In [79]:
pivot_df

,2024-01-07,2024-01-14,2024-01-21,2024-01-28,2024-02-04,2024-02-11,2024-02-18,2024-02-25,2024-03-03,2024-03-10,...,2025-10-26,2025-11-02,2025-11-09,2025-11-16,2025-11-23,2025-11-30,2025-12-07,2025-12-14,2025-12-21,2025-12-28
definition_id,,,,,,,,,,,,,,,,,,,,,
2,5088.18,9968.81,10046.68,5955.48,16212.31,14036.29,5.55,10175.46,1679.12,10290.37,...,2264.71,1431.12,4223.24,4397.45,0.00,2365.47,0.00,2208.73,1272.74,5400.91
3,-12002.60,-7501.15,-761.85,-2066.01,-16593.15,-3469.45,-1372.10,-4429.66,-6273.28,-14919.41,...,-16926.60,-4338.74,-2264.95,-6638.42,-17060.69,-5125.92,-6034.49,-6684.53,-18466.49,-11730.78
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,-5088.18,-9968.81,-10046.68,-5955.48,-16212.31,-14036.29,-5.55,-10175.46,-1679.12,-10290.37,...,-2264.71,-1431.12,-4223.24,-4397.45,0.00,-2365.47,0.00,-2208.73,-1272.74,-5400.91
12,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [80]:
# ********************** (2) Start working with totals based on totaled definitions *******************************
# Merge totals into totals definition
definition_totals_df = pd.merge(
    definition_tot_df,
    definition_totals_df,
    on="definition_id", how="left")
definition_totals_df.drop(columns=["id"], inplace=True)


In [81]:
definition_totals_df

,definition_id,key,definition_type,name,operator,definition_summarized
0,6,199,2.0,Bruto pamatdarbības naudas plūsma,+,2.0
1,6,199,2.0,Bruto pamatdarbības naudas plūsma,+,3.0
2,6,199,2.0,Bruto pamatdarbības naudas plūsma,+,4.0
3,6,199,2.0,Bruto pamatdarbības naudas plūsma,+,5.0
4,10,299,2.0,Pamatdarbības neto naudas plūsma,+,2.0
5,10,299,2.0,Pamatdarbības neto naudas plūsma,+,3.0
6,10,299,2.0,Pamatdarbības neto naudas plūsma,+,4.0
7,10,299,2.0,Pamatdarbības neto naudas plūsma,+,5.0
8,10,299,2.0,Pamatdarbības neto naudas plūsma,+,7.0
9,10,299,2.0,Pamatdarbības neto naudas plūsma,+,8.0


In [82]:
# Merge totals definition with summarized accounts pivot

merged_totals_df = pd.merge(definition_totals_df,
    pivot_df,
    left_on='definition_summarized',
    right_on='definition_id', 
    how='left'
)

# Add multiplication num for operator

merged_totals_df['op_num'] = merged_totals_df['operator'].map({'+': 1, '-': -1})

# Drop unnecessary columns

merged_totals_df.drop(columns = ['key', 'definition_type', 'name', 'operator', 'definition_summarized'], inplace=True)


In [83]:
merged_totals_df

,definition_id,2024-01-07 00:00:00,2024-01-14 00:00:00,2024-01-21 00:00:00,2024-01-28 00:00:00,2024-02-04 00:00:00,2024-02-11 00:00:00,2024-02-18 00:00:00,2024-02-25 00:00:00,2024-03-03 00:00:00,...,2025-11-02 00:00:00,2025-11-09 00:00:00,2025-11-16 00:00:00,2025-11-23 00:00:00,2025-11-30 00:00:00,2025-12-07 00:00:00,2025-12-14 00:00:00,2025-12-21 00:00:00,2025-12-28 00:00:00,op_num
0,6,5088.18,9968.81,10046.68,5955.48,16212.31,14036.29,5.55,10175.46,1679.12,...,1431.12,4223.24,4397.45,0.00,2365.47,0.00,2208.73,1272.74,5400.91,1.0
1,6,-12002.60,-7501.15,-761.85,-2066.01,-16593.15,-3469.45,-1372.10,-4429.66,-6273.28,...,-4338.74,-2264.95,-6638.42,-17060.69,-5125.92,-6034.49,-6684.53,-18466.49,-11730.78,1.0
2,6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.0
3,6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.0
4,10,5088.18,9968.81,10046.68,5955.48,16212.31,14036.29,5.55,10175.46,1679.12,...,1431.12,4223.24,4397.45,0.00,2365.47,0.00,2208.73,1272.74,5400.91,1.0
5,10,-12002.60,-7501.15,-761.85,-2066.01,-16593.15,-3469.45,-1372.10,-4429.66,-6273.28,...,-4338.74,-2264.95,-6638.42,-17060.69,-5125.92,-6034.49,-6684.53,-18466.49,-11730.78,1.0
6,10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.0
7,10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.0
8,10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.0
9,10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.0


In [84]:
# Choose value_columns for further summarization

value_columns = [col for col in merged_totals_df.columns if col not in ['definition_id', 'op_num']]

In [85]:
value_columns

[Timestamp('2024-01-07 00:00:00'),
 Timestamp('2024-01-14 00:00:00'),
 Timestamp('2024-01-21 00:00:00'),
 Timestamp('2024-01-28 00:00:00'),
 Timestamp('2024-02-04 00:00:00'),
 Timestamp('2024-02-11 00:00:00'),
 Timestamp('2024-02-18 00:00:00'),
 Timestamp('2024-02-25 00:00:00'),
 Timestamp('2024-03-03 00:00:00'),
 Timestamp('2024-03-10 00:00:00'),
 Timestamp('2024-03-17 00:00:00'),
 Timestamp('2024-03-24 00:00:00'),
 Timestamp('2024-03-31 00:00:00'),
 Timestamp('2024-04-07 00:00:00'),
 Timestamp('2024-04-14 00:00:00'),
 Timestamp('2024-04-21 00:00:00'),
 Timestamp('2024-04-28 00:00:00'),
 Timestamp('2024-05-05 00:00:00'),
 Timestamp('2024-05-12 00:00:00'),
 Timestamp('2024-05-19 00:00:00'),
 Timestamp('2024-05-26 00:00:00'),
 Timestamp('2024-06-02 00:00:00'),
 Timestamp('2024-06-09 00:00:00'),
 Timestamp('2024-06-16 00:00:00'),
 Timestamp('2024-06-23 00:00:00'),
 Timestamp('2024-06-30 00:00:00'),
 Timestamp('2024-07-07 00:00:00'),
 Timestamp('2024-07-14 00:00:00'),
 Timestamp('2024-07-

In [86]:
# Summarize value_columns based on group value and op_num

summarized_totals_df = (
    merged_totals_df.groupby('definition_id', group_keys=False)[value_columns + ['op_num']]
    .apply(lambda group: pd.Series(
        (group['op_num'].to_numpy()[:, None] * group[value_columns].to_numpy()).sum(axis=0),
        index=value_columns
    ))
    # .reset_index()
)


In [87]:
summarized_totals_df

,2024-01-07,2024-01-14,2024-01-21,2024-01-28,2024-02-04,2024-02-11,2024-02-18,2024-02-25,2024-03-03,2024-03-10,...,2025-10-26,2025-11-02,2025-11-09,2025-11-16,2025-11-23,2025-11-30,2025-12-07,2025-12-14,2025-12-21,2025-12-28
definition_id,,,,,,,,,,,,,,,,,,,,,
6,-6914.42,2467.66,9284.83,3889.47,-380.84,10566.84,-1366.55,5745.80,-4594.16,-4629.04,...,-14661.89,-2907.62,1958.29,-2240.97,-17060.69,-2760.45,-6034.49,-4475.80,-17193.75,-6329.87
10,-12002.60,-7501.15,-761.85,-2066.01,-16593.15,-3469.45,-1372.10,-4429.66,-6273.28,-14919.41,...,-16926.60,-4338.74,-2264.95,-6638.42,-17060.69,-5125.92,-6034.49,-6684.53,-18466.49,-11730.78
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
# ********************** (3) Start working with balances on end of each period *******************************

# Convert d_date to_datetime if not yet
cash_df['d_date'] = pd.to_datetime(cash_df['d_date'])


In [89]:
cash_df.head()

,id,d_id,d_type,d_number,d_date,d_vendor,d_customer,d_description,d_currency,gl_entry_type,gl_account,gl_amount,gl_amount_LC
0,7568,2284,1,1784,2020-01-01,NaN,45.0,Payment 1784,EUR,DR,2620,1349.31,1349.31
1,7550,2275,1,1775,2020-01-02,NaN,5.0,Payment 1775,EUR,DR,2620,1824.39,1824.39
2,16025,4713,2,3413,2020-01-02,98.0,NaN,Payment 3413,EUR,CR,2620,5267.30,5267.30
3,16959,5180,2,3880,2020-01-02,89.0,NaN,Payment 3880,EUR,CR,2620,753.54,753.54
4,15719,4560,2,3260,2020-01-04,83.0,NaN,Payment 3260,EUR,CR,2620,9757.83,9757.83


In [90]:
cash_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             2000 non-null   int64         
 1   d_id           2000 non-null   int64         
 2   d_type         2000 non-null   int64         
 3   d_number       2000 non-null   object        
 4   d_date         2000 non-null   datetime64[ns]
 5   d_vendor       1000 non-null   float64       
 6   d_customer     1000 non-null   float64       
 7   d_description  2000 non-null   object        
 8   d_currency     2000 non-null   object        
 9   gl_entry_type  2000 non-null   object        
 10  gl_account     2000 non-null   object        
 11  gl_amount      2000 non-null   float64       
 12  gl_amount_LC   2000 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int64(3), object(5)
memory usage: 203.2+ KB


In [91]:
# Initialize cumulative balance
cumulative_balance = 0  
balances = []

# Start index for efficient filtering
start_idx = 0  

for eoperiod in value_columns:
    # Filter only new transactions from the last weekend up to the current one
    new_transactions = cash_df[(cash_df.index >= start_idx) & (cash_df['d_date'] <= eoperiod)]
    
    # If there are new transactions, update cumulative balance
    if not new_transactions.empty:
        cumulative_balance += new_transactions.apply(
            lambda row: row['gl_amount_LC'] if row['gl_entry_type'] == 'DR' else -row['gl_amount_LC'], axis=1
        ).sum()
        
        # Move the start index forward to avoid redundant calculations
        start_idx = new_transactions.index[-1] + 1  

    # Store balance for the current weekend
    balances.append(cumulative_balance)



In [92]:
# Create a DataFrame with periods as columns
balances_df = pd.DataFrame([balances], columns=value_columns)


In [93]:
balances_df

,2024-01-07,2024-01-14,2024-01-21,2024-01-28,2024-02-04,2024-02-11,2024-02-18,2024-02-25,2024-03-03,2024-03-10,...,2025-10-26,2025-11-02,2025-11-09,2025-11-16,2025-11-23,2025-11-30,2025-12-07,2025-12-14,2025-12-21,2025-12-28
0,53985.82,65760.45,75017.74,81643.68,90092.09,102955.8,107578.91,115797.15,104985.4,95229.43,...,72632.33,59779.99,63844.97,73724.82,56150.69,62168.38,61401.94,68061.93,45843.37,43727.75


In [94]:
# Create a new DataFrame with repeated rows for each definition_id
balances_df = pd.merge(definition_bal_df["definition_id"], balances_df, how='cross')
balances_df = balances_df.set_index("definition_id")


In [95]:
balances_df

,2024-01-07,2024-01-14,2024-01-21,2024-01-28,2024-02-04,2024-02-11,2024-02-18,2024-02-25,2024-03-03,2024-03-10,...,2025-10-26,2025-11-02,2025-11-09,2025-11-16,2025-11-23,2025-11-30,2025-12-07,2025-12-14,2025-12-21,2025-12-28
definition_id,,,,,,,,,,,,,,,,,,,,,
30,53985.82,65760.45,75017.74,81643.68,90092.09,102955.8,107578.91,115797.15,104985.4,95229.43,...,72632.33,59779.99,63844.97,73724.82,56150.69,62168.38,61401.94,68061.93,45843.37,43727.75
31,53985.82,65760.45,75017.74,81643.68,90092.09,102955.8,107578.91,115797.15,104985.4,95229.43,...,72632.33,59779.99,63844.97,73724.82,56150.69,62168.38,61401.94,68061.93,45843.37,43727.75


In [96]:
# ********************** (3) Put together summarized accounts, totals and balances on end of each period *******************************

In [97]:
pivot_df.head()

,2024-01-07,2024-01-14,2024-01-21,2024-01-28,2024-02-04,2024-02-11,2024-02-18,2024-02-25,2024-03-03,2024-03-10,...,2025-10-26,2025-11-02,2025-11-09,2025-11-16,2025-11-23,2025-11-30,2025-12-07,2025-12-14,2025-12-21,2025-12-28
definition_id,,,,,,,,,,,,,,,,,,,,,
2,5088.18,9968.81,10046.68,5955.48,16212.31,14036.29,5.55,10175.46,1679.12,10290.37,...,2264.71,1431.12,4223.24,4397.45,0.00,2365.47,0.00,2208.73,1272.74,5400.91
3,-12002.60,-7501.15,-761.85,-2066.01,-16593.15,-3469.45,-1372.10,-4429.66,-6273.28,-14919.41,...,-16926.60,-4338.74,-2264.95,-6638.42,-17060.69,-5125.92,-6034.49,-6684.53,-18466.49,-11730.78
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [98]:
summarized_totals_df.head()

,2024-01-07,2024-01-14,2024-01-21,2024-01-28,2024-02-04,2024-02-11,2024-02-18,2024-02-25,2024-03-03,2024-03-10,...,2025-10-26,2025-11-02,2025-11-09,2025-11-16,2025-11-23,2025-11-30,2025-12-07,2025-12-14,2025-12-21,2025-12-28
definition_id,,,,,,,,,,,,,,,,,,,,,
6,-6914.42,2467.66,9284.83,3889.47,-380.84,10566.84,-1366.55,5745.80,-4594.16,-4629.04,...,-14661.89,-2907.62,1958.29,-2240.97,-17060.69,-2760.45,-6034.49,-4475.80,-17193.75,-6329.87
10,-12002.60,-7501.15,-761.85,-2066.01,-16593.15,-3469.45,-1372.10,-4429.66,-6273.28,-14919.41,...,-16926.60,-4338.74,-2264.95,-6638.42,-17060.69,-5125.92,-6034.49,-6684.53,-18466.49,-11730.78
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
balances_df.head()

,2024-01-07,2024-01-14,2024-01-21,2024-01-28,2024-02-04,2024-02-11,2024-02-18,2024-02-25,2024-03-03,2024-03-10,...,2025-10-26,2025-11-02,2025-11-09,2025-11-16,2025-11-23,2025-11-30,2025-12-07,2025-12-14,2025-12-21,2025-12-28
definition_id,,,,,,,,,,,,,,,,,,,,,
30,53985.82,65760.45,75017.74,81643.68,90092.09,102955.8,107578.91,115797.15,104985.4,95229.43,...,72632.33,59779.99,63844.97,73724.82,56150.69,62168.38,61401.94,68061.93,45843.37,43727.75
31,53985.82,65760.45,75017.74,81643.68,90092.09,102955.8,107578.91,115797.15,104985.4,95229.43,...,72632.33,59779.99,63844.97,73724.82,56150.69,62168.38,61401.94,68061.93,45843.37,43727.75


In [100]:
# Concatenate it all together
report_df = pd.concat([pivot_df, summarized_totals_df, balances_df])


In [101]:
report_df

,2024-01-07,2024-01-14,2024-01-21,2024-01-28,2024-02-04,2024-02-11,2024-02-18,2024-02-25,2024-03-03,2024-03-10,...,2025-10-26,2025-11-02,2025-11-09,2025-11-16,2025-11-23,2025-11-30,2025-12-07,2025-12-14,2025-12-21,2025-12-28
definition_id,,,,,,,,,,,,,,,,,,,,,
2,5088.18,9968.81,10046.68,5955.48,16212.31,14036.29,5.55,10175.46,1679.12,10290.37,...,2264.71,1431.12,4223.24,4397.45,0.00,2365.47,0.00,2208.73,1272.74,5400.91
3,-12002.60,-7501.15,-761.85,-2066.01,-16593.15,-3469.45,-1372.10,-4429.66,-6273.28,-14919.41,...,-16926.60,-4338.74,-2264.95,-6638.42,-17060.69,-5125.92,-6034.49,-6684.53,-18466.49,-11730.78
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,-5088.18,-9968.81,-10046.68,-5955.48,-16212.31,-14036.29,-5.55,-10175.46,-1679.12,-10290.37,...,-2264.71,-1431.12,-4223.24,-4397.45,0.00,-2365.47,0.00,-2208.73,-1272.74,-5400.91
12,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [102]:
# ********************** (4) Prepare report for visual appearance *******************************

# Merge report definition header with report
report_df = pd.merge(definition_df, report_df, left_on="definition_id", right_on="definition_id", how="left")

# Sort based on key value
report_df.sort_values("key", inplace=True)


In [103]:
report_df

,definition_id,key,definition_type,name,2024-01-07 00:00:00,2024-01-14 00:00:00,2024-01-21 00:00:00,2024-01-28 00:00:00,2024-02-04 00:00:00,2024-02-11 00:00:00,...,2025-10-26 00:00:00,2025-11-02 00:00:00,2025-11-09 00:00:00,2025-11-16 00:00:00,2025-11-23 00:00:00,2025-11-30 00:00:00,2025-12-07 00:00:00,2025-12-14 00:00:00,2025-12-21 00:00:00,2025-12-28 00:00:00
30,31,000,3.0,Naudas līdzekļu atlikums pārskata perioda beigās,53985.82,65760.45,75017.74,81643.68,90092.09,102955.80,...,72632.33,59779.99,63844.97,73724.82,56150.69,62168.38,61401.94,68061.93,45843.37,43727.75
0,1,100,NaN,Pamatdarbības naudas plūsma,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,5088.18,9968.81,10046.68,5955.48,16212.31,14036.29,...,2264.71,1431.12,4223.24,4397.45,0.00,2365.47,0.00,2208.73,1272.74,5400.91
2,3,102,1.0,Maksājumi piegādātājiem un darbuzņēmējiem,-12002.60,-7501.15,-761.85,-2066.01,-16593.15,-3469.45,...,-16926.60,-4338.74,-2264.95,-6638.42,-17060.69,-5125.92,-6034.49,-6684.53,-18466.49,-11730.78
3,4,103,1.0,Maksājumi darbiniekiem un pašnodarbinātajiem,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,6,199,2.0,Bruto pamatdarbības naudas plūsma,-6914.42,2467.66,9284.83,3889.47,-380.84,10566.84,...,-14661.89,-2907.62,1958.29,-2240.97,-17060.69,-2760.45,-6034.49,-4475.80,-17193.75,-6329.87
6,7,201,1.0,Izdevumi procentu maksājumiem,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,8,202,1.0,Izdevumi nodokļu maksājumiem,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,9,203,1.0,Naudas plūsma no ārkārtas posteņiem,-5088.18,-9968.81,-10046.68,-5955.48,-16212.31,-14036.29,...,-2264.71,-1431.12,-4223.24,-4397.45,0.00,-2365.47,0.00,-2208.73,-1272.74,-5400.91


In [104]:
# Subtract report formatting in separate dataframe
format_df = report_df["definition_type"]

In [105]:

format_df

30    3.0
0     NaN
1     1.0
2     1.0
3     1.0
4     1.0
5     2.0
6     1.0
7     1.0
8     1.0
9     2.0
10    NaN
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    2.0
20    NaN
21    1.0
22    1.0
23    1.0
24    1.0
25    1.0
26    2.0
27    1.0
28    2.0
29    3.0
Name: definition_type, dtype: float64

In [106]:
# Prepare report_df for visual appearance

report_df.drop(columns=["definition_id", "key", "definition_type"], inplace=True)
report_df.set_index("name", inplace=True)

# Format column headers to show only the date part
report_df.columns = [col.strftime(date_format()) if not pd.isnull(col) else col for col in report_df.columns]



In [107]:
report_df

,"07. Jan, 2024","14. Jan, 2024","21. Jan, 2024","28. Jan, 2024","04. Feb, 2024","11. Feb, 2024","18. Feb, 2024","25. Feb, 2024","03. Mar, 2024","10. Mar, 2024",...,"26. Oct, 2025","02. Nov, 2025","09. Nov, 2025","16. Nov, 2025","23. Nov, 2025","30. Nov, 2025","07. Dec, 2025","14. Dec, 2025","21. Dec, 2025","28. Dec, 2025"
name,,,,,,,,,,,,,,,,,,,,,
Naudas līdzekļu atlikums pārskata perioda beigās,53985.82,65760.45,75017.74,81643.68,90092.09,102955.80,107578.91,115797.15,104985.40,95229.43,...,72632.33,59779.99,63844.97,73724.82,56150.69,62168.38,61401.94,68061.93,45843.37,43727.75
Pamatdarbības naudas plūsma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ieņēmumi no preču un pakalpojumu pārdošanas,5088.18,9968.81,10046.68,5955.48,16212.31,14036.29,5.55,10175.46,1679.12,10290.37,...,2264.71,1431.12,4223.24,4397.45,0.00,2365.47,0.00,2208.73,1272.74,5400.91
Maksājumi piegādātājiem un darbuzņēmējiem,-12002.60,-7501.15,-761.85,-2066.01,-16593.15,-3469.45,-1372.10,-4429.66,-6273.28,-14919.41,...,-16926.60,-4338.74,-2264.95,-6638.42,-17060.69,-5125.92,-6034.49,-6684.53,-18466.49,-11730.78
Maksājumi darbiniekiem un pašnodarbinātajiem,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Pārējie pamatdarbības ieņēmumi un izdevumi,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Bruto pamatdarbības naudas plūsma,-6914.42,2467.66,9284.83,3889.47,-380.84,10566.84,-1366.55,5745.80,-4594.16,-4629.04,...,-14661.89,-2907.62,1958.29,-2240.97,-17060.69,-2760.45,-6034.49,-4475.80,-17193.75,-6329.87
Izdevumi procentu maksājumiem,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Izdevumi nodokļu maksājumiem,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [108]:
report_df.columns


Index(['07. Jan, 2024', '14. Jan, 2024', '21. Jan, 2024', '28. Jan, 2024',
       '04. Feb, 2024', '11. Feb, 2024', '18. Feb, 2024', '25. Feb, 2024',
       '03. Mar, 2024', '10. Mar, 2024',
       ...
       '26. Oct, 2025', '02. Nov, 2025', '09. Nov, 2025', '16. Nov, 2025',
       '23. Nov, 2025', '30. Nov, 2025', '07. Dec, 2025', '14. Dec, 2025',
       '21. Dec, 2025', '28. Dec, 2025'],
      dtype='object', length=104)